In [ ]:
# load package and fonts

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(font="Helvetica", rc={"figure.figsize":(4, 3)})
sns.set_theme(style="white")
sns.set_style('ticks')
sns.set_context("paper", font_scale=1.3, rc={"lines.linewidth": 1.75})

In [ ]:
# load data

report = pd.read_csv('../data/RRQ_tpch_end_to_end.csv', sep=';')

report = report.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
report.columns = report.columns.str.strip()


# processing list strings

def parse_list(s):
    if isinstance(s, str) and s.startswith('List'):
        return [float(x) for x in s[5: -1].split(",")]
    else:
        return s

report = report.applymap(parse_list)

report.loc[(report["viewConstraintFlag"] == "static fixed: 1.0") & (report["mechanism"] == "baseline"), "mechanism"] = "PrivateSQL"
report.loc[(report["viewConstraintFlag"] == "static fixed: 1.0") & (report["mechanism"] == "PrivateSQL"), "viewConstraintFlag"] = "dynamic fixed: 1.0"

report.head(5)

In [ ]:
# generate performance table

def process_for_runtime(df):
    ret = []
    mech_dt = ["aGM", "baseline", "PrivateSQL", "Chorus", "ChorusP"]
    for mech in mech_dt:
        filtered_df = df[df["mechanism"]==mech]
        if mech in ["aGM", "baseline", "PrivateSQL"]:
            ret.append(np.mean(filtered_df["setup_time"]))
        ret.append(np.mean(filtered_df["execution_time"]))
        ret.append(np.mean(filtered_df["utility"]))
        ret.append(np.mean(filtered_df["execution_time"]) / np.mean(filtered_df["utility"]))
    return ret


In [ ]:
fill_in = process_for_runtime(report)
print(fill_in)

latex_tab = """\\begin{{tabular}}{{ccccc}}
    \\toprule \hline
    Systems  & Setup Time   & Running Time & No. of Queries & Per Query Perf  \\\ \hline
    \oursystem     &  {:.2f} ms    &  {:.2f} ms  & {:.1f}  & {:.2f} ms \\\ 
    Vanilla &  {:.2f} ms    & {:.2f} ms  & {:.1f}   & {:.2f} ms  \\\ 
    sPrivateSQL &  {:.2f} ms &  {:.2f} ms & {:.1f} & {:.2f} ms \\\ 
    Chorus &  N/A   &  {:.2f} ms  & {:.1f}   & {:.2f} ms \\\ 
    ChorusP & N/A   &  {:.2f} ms  & {:.1f}  & {:.2f} ms \\\ \hline
    \\bottomrule
\end{{tabular}}""".format(fill_in[0], fill_in[1], fill_in[2], fill_in[3], fill_in[4], fill_in[5], fill_in[6], 
                        fill_in[7], fill_in[8], fill_in[9], fill_in[10], fill_in[11], fill_in[12], fill_in[13], 
                        fill_in[14], fill_in[15], fill_in[16], fill_in[17])
    
print(latex_tab)